参考: https://pytorch.org/tutorials/beginner/pytorch_with_examples.html

このチュートリアルではPyTorchの動かし方を学びます。  
PyTorchは2種類の特徴を持っています。  
<ul>
    <li>n次元のテンソルをGPUで動かせる</li>
    <li>ニューラルネットワークの構築と学習においての自動微分</li>
</ul>
このチュートリアルでは全結合のReluのネットワークを使います。ネットワークには隠れ層が1層あり、勾配降下法で最適化します。

# numpy

まずnumpyでネットワークを実装してみます。numpyはn次元のarrayのオブジェクトを使えます。numpyは数値計算にむいていますが、計算グラフやディープラーニングや勾配についての情報は持っていません。ただ2層くらいだったらnumpyでforwardとbackward込みで実装できます。  
まずはネットワークの構築です。

In [7]:
import numpy as np

# Nはバッチサイズ、D_inは入力の次元
# Hは隠れ層の次元、D_outは出力の次元
N,D_in,H,D_out=64,1000,100,10

# ランダムな入力と出力のデータを生成
x=np.random.randn(N,D_in) # (1000,)がバッチサイズ64個分
y=np.random.randn(N,D_out) # (10,)がバッチサイズ64個分

print("x = ",x.shape)
print("y = ",y.shape)

# 重みをランダムな値で初期化
w1=np.random.randn(D_in,H) # 1000→100
w2=np.random.randn(H,D_out) # 100→10

print("w1 = ",w1.shape)
print("w2 = ",w2.shape)

learning_rate=1e-6 # 学習率

x =  (64, 1000)
y =  (64, 10)
w1 =  (1000, 100)
w2 =  (100, 10)


それでは学習させてみます。

In [8]:
# 500エポック
for t in range(500):
    # forward: yの予測値を計算
    h=x.dot(w1) # h=w1*x
    h_relu=np.maximum(h,0) # h_relu=relu(h)
    y_pred=h_relu.dot(w2) # y_pred=w2*h_relu
    
    # ロスを計算して表示
    loss=np.square(y_pred-y).sum() # 二乗誤差を計算
    print(t,loss)
    
    # ロスをw1とw2の勾配を計算するために逆伝搬
    grad_y_pred=2.0*(y_pred-y)
    grad_w2=h_relu.T.dot(grad_y_pred)
    grad_h_relu=grad_y_pred.dot(w2.T)
    grad_h=grad_h_relu.copy()
    grad_h[h<0]=0
    grad_w1=x.T.dot(grad_h)
    
    # 重みを更新
    w1-=learning_rate*grad_w1 # w1=w1-lr*w1'
    w2-=learning_rate*grad_w2 # w2=w2-lr*w2'

0 31823478.64102552
1 28664697.11072056
2 27177728.48114389
3 23924352.459111504
4 18297027.783265993
5 12159365.726656396
6 7319922.7658959
7 4307388.4810226895
8 2638349.768839662
9 1748376.3967286015
10 1257545.3302723526
11 966869.5974510428
12 778260.087715298
13 645084.5712176858
14 544727.1464991323
15 465621.2507244223
16 401278.25149095536
17 347966.75784856256
18 303181.20164392306
19 265222.39391120384
20 232861.95343195298
21 205128.15383292665
22 181235.53536597226
23 160603.4135914872
24 142715.2989601657
25 127152.7975961942
26 113540.56395353696
27 101598.84457884528
28 91136.02799754421
29 81935.94878479112
30 73807.56925874762
31 66607.91961471579
32 60210.479892593095
33 54519.51930362925
34 49446.732255665
35 44917.74604277941
36 40863.89808077119
37 37224.12215310537
38 33950.73040594339
39 31004.115138373185
40 28348.85167781714
41 25950.341422123693
42 23779.9464973942
43 21815.09999841234
44 20032.8234273912
45 18415.178626433957
46 16944.791090433137
47 15607.0

422 0.00017791215546851644
423 0.00017057970294253686
424 0.00016355009253114577
425 0.00015681044280968742
426 0.00015034972495256603
427 0.00014415684471490303
428 0.00013821939868532274
429 0.000132527100842352
430 0.00012707004372643
431 0.00012183928619211875
432 0.00011682387750872615
433 0.000112015340018149
434 0.00010740523782888872
435 0.00010298880814873531
436 9.875297022913718e-05
437 9.46904208894794e-05
438 9.079554355535838e-05
439 8.706245705206053e-05
440 8.348241313541856e-05
441 8.004993038630952e-05
442 7.675917432104123e-05
443 7.360473398808896e-05
444 7.057953722367017e-05
445 6.767921202484466e-05
446 6.489861859383496e-05
447 6.223284537937133e-05
448 5.967634664272143e-05
449 5.722659601674396e-05
450 5.4877028625524724e-05
451 5.2624016823283655e-05
452 5.046339940095749e-05
453 4.839171410330933e-05
454 4.64054991852871e-05
455 4.450121263856558e-05
456 4.2675006136275984e-05
457 4.0923894829429454e-05
458 3.924503559527972e-05
459 3.763530237422944e-05
460

# Tensor

numpyは素晴らしいフレームワークですが、GPUの恩恵を受けられません。GPUを使うと50倍も早くなったりするので、是非使いたいです。  
PyTorchの根幹をなすのがテンソルです。PyTorchのテンソルはnumpyのarrayと同じです。テンソルはn次元のarrayで、PyTorchはこれを操作する関数があります。  
テンソルは計算グラフと勾配を見ることができます。またGPUも使えます。GPUでPyTorchを使いたい場合は、新しいデータのタイプに変換する必要があります。  
ここではPyTorchのテンソルで2層のネットワークを学習させます。numpyのときと同様に、forwardとbackwardのパスを実装する必要があります。

In [9]:
import torch

dtype=torch.float # データの型はfloat
device=torch.device("cpu") # CPUで実行
# device=torch.device("cuda:0") # GPUで実行

# Nはバッチサイズ、D_inは入力の次元
# Hは隠れ層の次元、D_outは出力の次元
N,D_in,H,D_out=64,1000,100,10

# ランダムな入力と出力のデータを生成する
x=torch.randn(N,D_in,device=device,dtype=dtype)
y=torch.randn(N,D_out,device=device,dtype=dtype)

# 重みをランダムに初期化
w1=torch.randn(D_in,H,device=device,dtype=dtype)
w2=torch.randn(H,D_out,device=device,dtype=dtype)

learning_rate=1e-6 # 学習率

for t in range(500): # 500エポック回す
    # forward: yの予測値を計算
    h=x.mm(w1) # h=w1*x
    h_relu=h.clamp(min=0) # h_relu=relu(h)
    y_pred=h_relu.mm(w2) # y_pred=w2*h_relu
    
    # ロスを計算して表示
    loss=(y_pred-y).pow(2).sum().item() # SE
    
    if t%100==99: # 100エポックごとに表示
        print(t,loss)
        
    # w1とw2の勾配を計算して逆伝搬
    grad_y_pred=2.0*(y_pred-y)
    grad_w2=h_relu.t().mm(grad_y_pred)
    grad_h_relu=grad_y_pred.mm(w2.t())
    grad_h=grad_h_relu.clone()
    grad_h[h<0]=0
    grad_w1=x.t().mm(grad_h)
    
    # 勾配降下法で重みを更新
    w1-=learning_rate*grad_w1
    w2-=learning_rate*grad_w2

99 914.9981689453125
199 14.325092315673828
299 0.40440312027931213
399 0.01374082826077938
499 0.000772079627495259


# Autograd

これまでの例では、ニューラルネットワークの順伝搬・逆伝搬ともに自分で実装しなければなりませんでした。ネットワークが大きくなると自力でやるのは辛いです。  
ありがたいことに、ニューラルネットワークには自動微分の仕組みがあります。PyTorchではautogradのパッケージが使えます。autogradを使うと、forwardで計算グラフを定義します。各ノードはテンソルで、各エッジは入力のテンソルから出力のテンソルを作る関数となります。このグラフを作れば逆伝搬で簡単に勾配を計算できます。  
xをテンソルとしたとき、x.requires_grad=Trueとするとx.gradがxの勾配を持つことになります。  
それではPyTorchのテンソルとautogradを使って2層のネットワークを構成します。autogradを利用すると、backwardを実装する必要がなくなります。

In [11]:
import torch

dtype=torch.float # データ型はfloat
device=torch.device("cpu") # CPUの場合
# device=torch.device("cuda:0") # GPUの場合

# 入力と出力を持つランダムなテンソルを生成する
# requires_grad=Falseとすると、backwardで勾配を計算しないということになる
# 入力、出力なので勾配を持つ必要はない
x=torch.randn(N,D_in,device=device,dtype=dtype,requires_grad=False)
y=torch.randn(N,D_out,device=device,dtype=dtype,requires_grad=False)

# ランダムな値を持つ重みのテンソルを作成
# requires_grad=Trueとすることでbackwardで勾配を計算することとする
w1=torch.randn(D_in,H,device=device,dtype=dtype,requires_grad=True)
w2=torch.randn(H,D_out,device=device,dtype=dtype,requires_grad=True)

learning_rate=1e-6 # 学習率

for t in range(500): # エポックで繰り返す
    # forwardでは、yの予測値を計算する
    # 以前とは違い、逆伝搬のために中間の値を持っておく必要はない
    
    # (N,D_in)×(D_in,H)→(N,H), (N,H)×(H,D_out)→(N,D_out)
    # clamp()は値を範囲内に変更する関数、ここではrelu
    y_pred=x.mm(w1).clamp(min=0).mm(w2) 
    
    # ロスを計算して表示する
    # ロスは(1,)のテンソルに入っている
    # loss.item()でスカラーを取れる
    loss=(y_pred-y).pow(2).sum()
    
    if t%100==99: # 100回ごとに表示
        print(t,loss.item())
        
    # backwardはrequires_grad=Trueとしたおかげで1行ですむ
    # requires_grad=Trueとした全てのテンソルについて、ロスの勾配を計算する
    # この後にw1.grad,w2.gradでそれぞれ勾配を取得できる
    loss.backward()
    
    # 勾配降下法で重みを更新する
    # torch.no_grad()の中で実行することで、勾配の計算に影響を与えないようにする
    # weight.dataとweight.grad.dataで実装する方法もある
    # tensor.dataはtensorと同じデータを返すが、勾配に影響を与えない
    # torch.optimでもできる
    with torch.no_grad():
        w1-=learning_rate*w1.grad # w1の更新
        w2-=learning_rate*w2.grad # w2の更新
        
        # 重みを更新した後に、手動で勾配をゼロにする
        w1.grad.zero_()
        w2.grad.zero_()

99 567.8706665039062
199 2.621206045150757
299 0.018998155370354652
399 0.0003641570801846683
499 5.5538352171424776e-05


# autogradの関数を自分で定義

autogradは操作するのは2つだけです。forward()は入力のテンソルから出力のテンソルを計算します。backward()では出力のテンソルの勾配を受け取って、入力のテンソルの勾配を計算します。  
PyTorchではtorch.autograd.Functionのサブクラスを定義してforward()とbackward()を定義することで自作のautogradを定義できます。そして入力となるテンソルを渡してインスタンスを構築して関数として呼ぶことで使用することができます。  
ここではReLUの非線形性に対応したautogradを自作し、2層のニューラルネットワークで利用します。

In [12]:
import torch

class MyReLU(torch.autograd.Function):
    # torch.autograd.Functionをサブクラスとして、forward()とbackward()を実装することで自作のautogradの関数を作ることができる
    
    @staticmethod
    def forward(ctx,input):
        # forwardでは入力としてテンソルを受け取り、テンソルを出力する
        # ctxはbackwardの計算のために情報を保持するcontextのオブジェクト
        # ctx.save_for_backward()で任意のオブジェクトを、backwardの計算のために保持できる
        ctx.save_for_backward(input)
        
        return input.clamp(min=0)
    
    @staticmethod
    def backward(ctx,grad_output):
        # backwardではロスの勾配を含むテンソルを受け取る
        input,=ctx.saved_tensors
        grad_input=grad_output.clone()
        grad_input[input<0]=0
        
        return grad_input

それではネットワークを構築します。

In [14]:
dtype=torch.float
device=torch.device("cpu") # CPUで実行
# device=torch.device("cuda:0") # GPUで実行

# Nはバッチサイズ、D_inは入力の次元
# Hは隠れ層の次元、D_outは出力の次元
N,D_in,H,D_out=64,1000,100,10

# 入力と出力のためにランダムな値を持つテンソルを作る
x=torch.randn(N,D_in,device=device,dtype=dtype)
y=torch.randn(N,D_out,device=device,dtype=dtype)

# 重みのテンソルをランダムな値で初期化
w1=torch.randn(D_in,H,device=device,dtype=dtype,requires_grad=True)
w2=torch.randn(H,D_out,device=device,dtype=dtype,requires_grad=True)

learning_rate=1e-6 # 学習率

for t in range(500): # エポック
    # 自作の関数を適用するために、Function.applyを使う
    relu=MyReLU.apply
    
    # forward: yの予測値を計算する
    # 自作のautogradでReLUを計算する
    y_pred=relu(x.mm(w1)).mm(w2)
    
    # ロスを計算して表示する
    loss=(y_pred-y).pow(2).sum() # SE
    
    if t%100==99: # 100エポックごとに表示
        print(t,loss.item())
        
    # autogradを使ってbackwardを計算する
    loss.backward()
        
    # 勾配降下法で重みを更新する
    with torch.no_grad():
        w1-=learning_rate*w1.grad
        w2-=learning_rate*w2.grad
        
        # 重みを更新した後に勾配を手動で0にする
        w1.grad.zero_()
        w2.grad.zero_()

99 425.3764953613281
199 1.2911951541900635
299 0.005374168511480093
399 0.00014156590623315424
499 3.016583650605753e-05


# nnモジュール

計算グラフとautogradの組み合わせはとても有用ですが、大きいニューラルネットワークに対してはautogradはきついです。  
ニューラルネットワークを構築する際は、計算をレイヤに落とし込むことが多いです。このレイヤは学習可能なパラメータも持っています。  
TensorFlowでは、Keras、Tensorflow-Slim、TFlLearnといった、生の計算グラフに適用できる上位の概念がありました。  
PyTorchでは、上に挙げたものをnnパッケージで実現できます。nnパッケージにはモジュールがあり、これがニューラルネットワークのレイヤに相当します。  
モジュールではテンソルを入力とし、同じくテンソルを出力しますが、学習可能なパラメータをもつこともあります。  
nnパッケージにはロス関数もあります。  
ここでは2層のニューラルネットワークをnnパッケージを使って実装してみます。

In [15]:
import torch

# Nはバッチサイズ、D_inは入力の次元
# Hは隠れ層の次元、D_outは出力の次元
N,D_in,H,D_out=64,1000,100,10

# 入力と出力を持つテンソルを作成し、ランダムな値で初期化する
x=torch.randn(N,D_in) # 入力
y=torch.randn(N,D_out) # 出力

# nnパッケージを使って、モデルをレイヤのシーケンスと捉えて実装する
# nn.Sequentialは入力をシーケンスのように処理して出力する
# 各Linearモジュールは線形関数を使って入力から出力を計算し、重みとバイアスの内部のテンソルを計算する
model=torch.nn.Sequential(
    torch.nn.Linear(D_in,H), # 線形
    torch.nn.ReLU(), # ReLU
    torch.nn.Linear(H,D_out) # 線形
)

# nnパッケージにはロス関数もある
# ここではMSEを使う
loss_fn=torch.nn.MSELoss(reduction="sum") # MSE

learning_rate=1e-4 # 学習率

for t in range(500): # エポック
    # forward: モデルにxを入力することでyの予測値を計算する
    # Moduleオブジェクトは__call__をオーバーライドするので、関数のように呼び出すことができる
    # そのときはModuleにテンソルを入れて、出力としてテンソルを得られる
    y_pred=model(x) # ネットワークの出力を取得する
    
    # ロスを計算して表示する
    # yの実測値と予測値を入れることで、ロスを取得できる
    loss=loss_fn(y_pred,y) # ロス
    
    if t%100==99: # 100エポックごとに表示
        print(t,loss.item())
    
    # backward: ロスの勾配を学習可能なパラメータについて計算する
    # 内部的に、Moduleのパラメータはrequires_grad=Trueとして持っている
    # これでモデルの全ての学習可能なパラメータのために勾配を計算できる
    loss.backward() # 逆伝搬
    
    # 勾配降下法で重みを更新する
    # 各パラメータはテンソルなので、これまでの方法で各勾配にアクセスできる
    with torch.no_grad(): # 勾配を蓄積しない
        for param in model.parameters(): # 各パラメータについて
            param-=learning_rate*param.grad # パラメータを更新する

99 92.52568054199219
199 41.80598068237305
299 32.95942306518555
399 32.760963439941406
499 42.08575439453125


# optim

これまで、重みを手動で更新する方法を見てきました。torch.no_grad()や.dataを使うことで勾配を蓄積せずに進める必要がありました。  
最適化関数がSGDとかシンプルなものだったらこれで良いのですが、AdaGrad、RMSProp、Adamなどのもっと複雑なものを使う際にはきついです。  
optimパッケージには最適化関数のアルゴリズムのアイデアが取り入れられていて、一般的な最適化関数をカバーしています。  
ここでは以前nnパッケージで構築したモデルを、optimパッケージのAdamアルゴリズムで最適化します。

In [16]:
import torch

# Nはバッチサイズ、D_inは入力の次元
# Hは隠れ層の次元、D_outは出力の次元
N,D_in,H,D_out=64,1000,100,10

# 入力と出力の情報を持つテンソルを作り、ランダムな値で初期化する
x=torch.randn(N,D_in) # 入力
y=torch.randn(N,D_out) # 出力

# nnパッケージを使ってモデルとロス関数を定義する
model=torch.nn.Sequential(
    torch.nn.Linear(D_in,H),
    torch.nn.ReLU(),
    torch.nn.Linear(H,D_out)
)

loss_fn=torch.nn.MSELoss(reduction="sum") # ロス関数

# optimパッケージで、モデルの重みを最適化する関数を定義する
# ここではAdamを使う
# optimパッケージにはいろんな最適化関数がある
# Adam()の一つ目の引数には更新するテンソルを入れる
learning_rate=1e-4 # 学習率
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate) # 最適化関数

for t in range(500): # エポック
    # forward: モデルにxを入れてyの予測値を計算する
    y_pred=model(x) # モデルに入れて出力する
    
    loss=loss_fn(y_pred,y) # ロス
    
    if t%100==99: # 100エポックごとにロスを表示する
        print(t,loss.item())
    
    # backward()の前にoptimizer.zero_grad()を実行する
    # そうしないと前のエポックで計算した勾配にどんどん蓄積されていってしまう
    optimizer.zero_grad()
    
    # backward: ロスの勾配を計算する
    loss.backward()
    
    # Optimizerのstep()でパラメータの更新を行う
    optimizer.step()

99 39.05604934692383
199 0.4798465669155121
299 0.0018017084803432226
399 3.6764358810614794e-06
499 5.767328836725483e-09


# nnモジュールでクラスを自作

sequentialよりも複雑なネットワークを作りたいときは、nn.Moduleをサブクラスにしてforward()を実装します。  
このforward()では入力としてテンソルを受け、テンソルを出力とします。  
ここでは、下のようにサブクラスを使って2層のネットワークを作ります。

In [ ]:
import torch

class TwoLayerNet(torch.nn.Module):
    def __init__(self,D_in,H,D_out):
        # ここではnn.Linearモジュールを2つインスタンス化して、変数として持つ
        super(TwoLayerNet,self).__init__()
        self.linear1=torch.nn.Linear(D_in,H) # 第1層
        self.linear2=torch.nn.Linear(H,D_out) # 第2層
        
    def forward(self,x):
        # forward()ではテンソルを入力として、テンソルを出力する
        # __init__()で定義したモジュールも使うことができる
        h_relu=self.linear1(x).clamp(min=0) # relu
        y_pred=self.linear2(h_relu) # 予測値を算出
        
        return y_pred

In [17]:
# Nはバッチサイズ、D_inは入力の次元
# Hは隠れ層の次元、D_outは出力の次元
N,D_in,H,D_out=64,1000,100,10

# 入力と出力のテンソル
x=torch.randn(N,D_in)
y=torch.randn(N,D_out)

# モデルを構築する
model=TwoLayerNet(D_in,H,D_out)

# ロス関数と最適化関数を定義する
# model.parameters()でモデルの中の学習可能なパラメータを取得できる
criterion=torch.nn.MSELoss(reduction="sum")
optimizer=torch.optim.SGD(model.parameters(),lr=1e-4)

for t in range(500): # エポック
    # forward: モデルでyの予測値を計算する
    y_pred=model(x)
    
    # ロスを計算して表示する
    loss=criterion(y_pred,y)
    
    if t%100==99: # 100エポックごとに表示
        print(t,loss.item())
        
    # 勾配を0にし、逆伝搬させ、重みを更新する
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 3.0537233352661133
199 0.03936923295259476
299 0.0008906815201044083
399 3.08222442981787e-05
499 1.5793425518495496e-06


# Control Flow + Weight Sharing

動的グラフとweight sharingの例として、変なネットワークを構成します。ここでは、全結合でReLUを使います。  
各層では1-4のランダムな値を取得し、その値に応じた回数だけ隠れ層で同じ重みを何度も使いながら再び隠れ層の計算をします。  
このモデルではループを構成するのに普通のPythonのflow controlが使えます。  
weight sharingはnn.Moduleで実装できます。  
こんな感じです。

In [21]:
import random
import torch

class DynamicNet(torch.nn.Module):
    def __init__(self,D_in,H,D_out):
        # ここではforward()で使うnn.Linearのインスタンスを3つ用意する
        super(DynamicNet,self).__init__()
        self.input_linear=torch.nn.Linear(D_in,H)
        self.middle_linear=torch.nn.Linear(H,H)
        self.output_linear=torch.nn.Linear(H,D_out)
        
    def forward(self,x):
        # 0-3からランダムな整数を取得する
        # middle_linearを何回も計算する
        # forwardでは動的な計算グラフを構築するので、Pythonのcontrol-flowの枠組みでループを構成する
        # 同じModuleを何回も使っても大丈夫
        # この点がLua Torchからの改善点
        h_relu=self.input_linear(x).clamp(min=0) # relu
        
        for _ in range(random.randint(0,3)): # 0-3回、さらなる隠れ層の計算をする
            h_relu=self.middle_linear(h_relu).clamp(min=0)
    
        y_pred=self.output_linear(h_relu) # 出力層を計算
        
        return y_pred 

それでは学習させます。

In [22]:
# Nはバッチサイズ、D_inは入力の次元
# Hは隠れ層の次元、D_outは出力の次元
N,D_in,H,D_out=64,1000,100,10

# 入力と出力
x=torch.randn(N,D_in)
y=torch.randn(N,D_out)

# モデルを構築
model=DynamicNet(D_in,H,D_out)

# ロス関数と最適化関数を定義する
# このモデルをSGDで学習させるのはきついので、momentumを使う
criterion=torch.nn.MSELoss(reduction="sum")
optimizer=torch.optim.SGD(model.parameters(),lr=1e-4,momentum=0.9)

for t in range(500):
    # forward: yの予測値を計算する
    y_pred=model(x)
    
    # ロスを計算して表示する
    loss=criterion(y_pred,y)
    
    if t%100==99: # 100エポックごとに表示する
        print(t,loss.item())
        
    # 勾配をゼロにし、逆伝搬させて、重みを更新する
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 21.761653900146484
199 6.2540283203125
299 1.6713652610778809
399 0.5263149738311768
499 0.2223612666130066
